In [1]:
print("Test Scucessfully!")

Test Scucessfully!


# Load Dataset and Model

In [2]:
from datasets import load_dataset

# Load Dataset
dataset_name = "glue"
task_name = "sst2"
dataset = load_dataset(dataset_name, task_name, split="train")
validation = load_dataset(dataset_name, task_name, split="validation")

Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Version 2-7b
# base_model_name = "meta-llama/Llama-2-7b-hf"
# Version 2-13b-chat
base_model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-13b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

# Inference

### CHAT INFERENCE MODE

In [18]:
dialogs = """\
<s>[INST] <<SYS>>
You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.
<</SYS>>

Here is an inference example:
Sentence: I am so happy today.
Sentiment: [/INST] positive </s><s>[INST] Here is another inference example:
Sentence: I am so sad today.
Sentiment: [/INST] negative </s><s>[INST] Please classify the sentiment of the following sentence into positive or negative:
Sentence: it 's a charming and often affecting journey .
Sentiment: [/INST] 
"""


In [19]:
inputs = tokenizer(dialogs, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST] <<SYS>>
You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.
<</SYS>>

Here is an inference example:
Sentence: I am so happy today.
Sentiment: [/INST] positive  [INST] Here is another inference example:
Sentence: I am so sad today.
Sentiment: [/INST] negative  [INST] Please classify the sentiment of the following sentence into positive or negative:
Sentence: it 's a charming and often affecting journey .
Sentiment: [/INST] 
Positive


### Test Inference Mode 1

In [13]:
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative."
sentence = "It really sucks that I have to do this."
examples = """
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative
"""

# text = system_prompt + "\n" + user_msg_1 + "\n" + "Sentiment:"
text = system_prompt + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment: "

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s> You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative

Sentence: It really sucks that I have to do this.
Sentiment:  negative

Sentence: I am so excited to go on vacation.
Sentiment: positive

Now, please classify the sentiment of the following message:

Sentence: I am so frustrated with this stupid app.

Please select one of the following options:

A) positive
B) negative

Please select one of the options based on the sentiment of the message.</s>


### Inference Mode 2

In [27]:
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative."
sentence = "unflinchingly bleak and desperate"

# text = system_prompt + "\n" + user_msg_1 + "\n" + "Sentiment:"
text = system_prompt + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment: "

pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=100)
result = pipe(f"{text}")
print(result[0]['generated_text'])

Input length of input_ids is 110, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative


Sentence: unflinchingly bleak and desperate
Sentiment:  negative


## Few Inference with text-generation

In [14]:
# System Prompt
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative."
# User Prompt
user_prompt = "Please classify the sentiment of the following sentence into positive or negative:"
# Examples
examples = """
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative

"""

In [17]:
import re
from tqdm import tqdm

def evaluate(dataset):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }

    compared_result = []

    for i, val in enumerate(tqdm(dataset)):
        label_text = label_map[val['label']]
        sentence = val['sentence']

        # Make input
        text = user_prompt + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment: "
        inputs = tokenizer(text, return_tensors="pt").to("cuda")

        # Generate
        outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)
        # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
        outputs_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(outputs_text)
        if i == 2:
            break

        matches = re.findall(r"Sentiment: (.+)", outputs_text)
        selected_sentiment = matches[2].strip() if len(matches) >= 3 else None
        # print(selected_sentiment)

        # Compare prediction and label
        if selected_sentiment == label_text:
            compared_result.append(1)
        else:
            compared_result.append(0)

    return compared_result


In [18]:
result_list = evaluate(validation)

  0%|          | 1/872 [00:00<03:42,  3.92it/s]

Please classify the sentiment of the following sentence into positive or negative:
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative


Sentence: it 's a charming and often affecting journey . 
Sentiment:  positive


  0%|          | 2/872 [00:04<40:03,  2.76s/it]

Please classify the sentiment of the following sentence into positive or negative:
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative


Sentence: unflinchingly bleak and desperate 
Sentiment:  negative

Sentence: unflinchingly hopeful and optimistic 
Sentiment: positive

Please classify the sentiment of the following sentence into positive or negative:
Sentence: unflinchingly bleak and desperate.

Please select one of the following options:

A) positive
B) negative
C) neutral

Please


  0%|          | 2/872 [00:05<36:22,  2.51s/it]

Please classify the sentiment of the following sentence into positive or negative:
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative


Sentence: allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
Sentiment:  positive


In [11]:
print("Accuracy:", result_list.count(1)/len(result_list))

Accuracy: 0.8623853211009175


# Few-shot Inference with chat-generation (LangChain)

In [20]:
from langchain.prompts.prompt import PromptTemplate

# Define the system prompt
system_prompt_template = "<<SYS>>\n{system_prompt_text}\n<</SYS>>\n\n"
system_prompt_text = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: 'positive' or 'negative'."
system_prompt = system_prompt_template.format(system_prompt_text=system_prompt_text)

# define few-shot inference instructions
inf_instruction1 = "Here is an inference example:\n"
inf_instruction2 = "Here is another inference example:\n"

# Define the example prompts
example_prompts = [
    {"system_prompt": inf_instruction1, "sentence": "I am so happy today.", "sentiment": "positive"},
    # {"system_prompt": inf_instruction2, "sentence": "They are really friendly.", "sentiment": "positive"},
    {"system_prompt": inf_instruction2, "sentence": "I am so sad today.", "sentiment": "negative"},
    # {"system_prompt": inf_instruction2, "sentence": "I hate spiders, they are creepy and scary.", "sentiment": "negative"}
]

# Define the format of the instance prompt
inst_format = "<s>[INST] {system_prompt}Sentence: {sentence}\nSentiment: [/INST] {sentiment} </s>"

# Define the format of the inference prompt
inference_format = "<s>[INST] {system_prompt}Sentence: {sentence}\nSentiment: [/INST] \n"

# Start constructing the prompts
# prompts = "<s>[INST] <<SYS>>\n" + system_prompt + "\n<</SYS>>\n\n"
prompts = ""

# Add the example prompts
for example in example_prompts:
    prompts += inst_format.format(**example)

# Your final prompts look like this:
print(prompts)


<s>[INST] Here is an inference example:
Sentence: I am so happy today.
Sentiment: [/INST] positive </s><s>[INST] Here is another inference example:
Sentence: I am so sad today.
Sentiment: [/INST] negative </s>


In [21]:
inference_prompt = "Please classify the sentiment of the following sentence into one of the following categories:'positive' or 'negative':\n"
# inference_prompt = "Please tell me the following sentence is positive or negative:\n"
# There is a space at the end of the sst2 sentence, so we need to remove it
data = {"system_prompt": inference_prompt, "sentence": validation[20]['sentence'][:-1], "sentiment": ""}
try_p = prompts + inference_format.format(**data)

In [22]:
print(try_p)

<s>[INST] Here is an inference example:
Sentence: I am so happy today.
Sentiment: [/INST] positive </s><s>[INST] Here is another inference example:
Sentence: I am so sad today.
Sentiment: [/INST] negative </s><s>[INST] Please classify the sentiment of the following sentence into one of the following categories:'positive' or 'negative':
Sentence: pumpkin takes an admirable look at the hypocrisy of political correctness , but it does so with such an uneven tone that you never know when humor ends and tragedy begins .
Sentiment: [/INST] 



In [23]:
import string
def remove_last_punctuation(text):
    punctuation_chars = set(string.punctuation)

    selected_sentiment = text.split("\n")[-1].lower()

    # If the last character is a punctuation, remove it
    while selected_sentiment and selected_sentiment[-1] in punctuation_chars:
        selected_sentiment = selected_sentiment[:-1]

    return selected_sentiment


In [24]:
import re
from tqdm import tqdm

def evaluate_chat(dataset, prefix_prompt, inference_format, user_prompt):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }

    compared_result = []
    invalid_index = []

    for i, val in enumerate(tqdm(dataset)):
        label_text = label_map[val['label']]
        sentence = val['sentence'][:-1]
        sent_dict = {
            "system_prompt": user_prompt,
            "sentence": sentence,
            "sentiment": ""
        }

        # Make input
        text = prefix_prompt + inference_format.format(**sent_dict)
        inputs = tokenizer(text, return_tensors="pt").to("cuda")

        # Generate
        outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)
        # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
        outputs_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # print(outputs_text)
        selected_sentiment = outputs_text.split("\n")[-1].lower()
        selected_sentiment = remove_last_punctuation(selected_sentiment)
        # print(selected_sentiment)
        # if i == 21:
        #     break

        # Abnormal case
        if selected_sentiment not in ['positive', 'negative']:
            invalid_index.append(i)
            compared_result.append(0)
            continue

        # Compare prediction and label
        # assert selected_sentiment in ['positive', 'negative'], f"Prediction {i} is not valid: {selected_sentiment}"
        if selected_sentiment == label_text:
            compared_result.append(1)
        else:
            compared_result.append(0)

    return compared_result, invalid_index


In [25]:
res_chat, invalid_index = evaluate_chat(validation, prompts, inference_format, inference_prompt)

  0%|          | 0/872 [00:00<?, ?it/s]

100%|██████████| 872/872 [09:03<00:00,  1.60it/s]


In [248]:
print("Accuracy:", res_chat.count(1)/len(res_chat))

Accuracy: 0.8222477064220184


In [235]:
len(res_chat)

872

In [249]:
len(invalid_index)

103